# 1.2 Niveles Lingüísticos II

<a target="_blank" href="https://colab.research.google.com/github/umoqnier/cl-2026-2-lab/blob/main/notebooks/1_2_niveles_linguisticos.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Objetivos

- Trabajar tareas a diferentes niveles lingüísticos (Fonético, Morfólogico, Sintáctico)
- Manipular y recuper información de datasets disponibles en Github para resolver tareas de NLP
- Comparar enfoques basados en reglas y estadísticos para el análisis morfológico

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/IPA_chart_2020.svg/660px-IPA_chart_2020.svg.png" width=500></center

In [ ]:
from IPython.display import YouTubeVideo

In [ ]:
YouTubeVideo("DcNMCB-Gsn8", width=960, height=615)

In [ ]:
YouTubeVideo("74nnLh0Vdcc", width=960, height=615)

In [3]:
import http
import re
from collections import defaultdict

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import requests as r
from nltk.corpus import cess_esp
from rich import print as rprint
from rich.columns import Columns
from rich.panel import Panel
from rich.rule import Rule
from rich.text import Text
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF

### International Phonetic Alphabet (IPA)

- Las lenguas naturales tienen muchos sonidos diferentes por lo que necesitamos una forma de describirlos independientemente de las lenguas
- IPA es una representación escrita de los [sonidos](https://www.ipachart.com/) del [habla](http://ipa-reader.xyz/)

### Dataset: [IPA-dict](https://github.com/open-dict-data/ipa-dict) de open-dict

- Diccionario de palabras para varios idiomas con su representación fonética
- Representación simple, una palabra por renglon con el formato:

```
[PALABRA][TAB][IPA]

Ejemplos
mariguana	/maɾiɣwana/
zyuganov's   /ˈzjuɡɑnɑvz/, /ˈzuɡɑnɑvz/
```

- [ISO language codes](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes)
- URL: `https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/<iso-lang>`

#### Explorando el corpus 🗺️

In [4]:
IPA_URL = "https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/{lang}.txt"

In [8]:
# ¿Como empezamos?
response = r.get(IPA_URL.format(lang="es_MX"))

In [16]:
ipa_list = response.text[:1000].split("\n")

In [17]:
ipa_list[-1].split("\t")

['ababillaréis', '/aβaβiʎaˈɾe']

#### Obtención y manipulación

In [18]:
def download_ipa_corpus(iso_lang: str) -> str:
    """Get ipa-dict file from Github

    Parameters:
    -----------
    iso_lang:
        Language as iso code

    Results:
    --------
    dict:
        Dictionary with words as keys and phonetic representation
        as values for a given lang code
    """
    print(f"Downloading {iso_lang}", end="::")
    response = r.get(IPA_URL.format(lang=iso_lang))
    status_code = response.status_code
    print(f"status={status_code}")
    if status_code != http.HTTPStatus.OK:
        print(f"ERROR on {iso_lang} :(")
        return ""
    return response.text

In [19]:
download_ipa_corpus("ar")[:100]

'آئل\t/aːʔil/\nآب\t/aːb/, /aːba/\nآباء\t/aːbaːʔ/\nآباد\t/aːbaːd/\nآبار\t/aːbaːr/\nآباض\t/aːbaːdˤ/\nآباط\t/aːbaːtˤ/'

In [20]:
def parse_response(response: str) -> dict:
    """Parse text response from ipa-dict to python dict

    Each row have the format:
    [WORD][TAB]/[IPA]/(, /[IPA]/)?

    Parameters
    ----------
    response: str
        ipa-dict raw text

    Returns
    -------
    dict:
        A dictionary with the word as key and the phonetic
        representations as value
    """
    ipa_list = response.rstrip().split("\n")
    result = {}
    for item in ipa_list:
        if item == '':
            continue
        item_list = item.split("\t")
        result[item_list[0]] = item_list[1]
    return result

In [24]:
es_data = parse_response(download_ipa_corpus("es_MX"))

In [22]:
def get_ipa_transcriptions(word: str, dataset: dict) -> list[str]:
    """Search for a word in an IPA phonetics dict

    Given a word this function return the IPA transcriptions

    Parameters:
    -----------
    word: str
        A word to search in the dataset
    dataset: dict
        A dataset for a given language code

    Returns
    -------
    list[str]:
        List with posible transcriptions if any,
        else an empty list
    """
    return dataset.get(word.lower(), "").split(", ")

In [27]:
get_ipa_transcriptions("mayonesa", es_data)

['/maʝonesa/']

#### Obtengamos datasets

In [28]:
# Get datasets
dataset_es_mx = parse_response(download_ipa_corpus("es_MX"))
dataset_ja = parse_response(download_ipa_corpus("ja"))
dataset_en_us = parse_response(download_ipa_corpus("en_US"))
dataset_fr = parse_response(download_ipa_corpus("fr_FR"))

In [29]:
# Simple query
get_ipa_transcriptions("beautiful", dataset_en_us)

['/ˈbjutəfəɫ/']

In [30]:
# Examples
print(f"dog -> {get_ipa_transcriptions('dog', dataset_en_us)} 🐶")
print(f"mariguana -> {get_ipa_transcriptions('mariguana', dataset_es_mx)} 🪴")
print(f"猫 - > {get_ipa_transcriptions('猫', dataset_ja)} 🐈")
print(f"croissant -> {get_ipa_transcriptions('croissant', dataset_fr)} 🥐")

dog -> ['/ˈdɔɡ/'] 🐶
mariguana -> ['/maɾiɣwana/'] 🪴
猫 - > ['/neko/', '/nekoma/'] 🐈
croissant -> ['/kʁwasɑ̃/'] 🥐


In [31]:
# Diferentes formas de pronunciar
print(f"[es_MX] hotel | {dataset_es_mx['hotel']}")
print(f"[en_US] hotel | {dataset_en_us['hotel']}")

[es_MX] hotel | /otel/
[en_US] hotel | /hoʊˈtɛɫ/


In [32]:
print(f"[ja] ホテル | {dataset_ja['ホテル']}")
print(f"[fr] hôtel | {dataset_fr['hôtel']}")

[ja] ホテル | /hoteɾɯ/
[fr] hôtel | /otɛl/


#### Obtener la distribución de frecuencias de los símbolos fonológicos para español

In [35]:
def get_phone_symbols_freq(dataset: dict):
    freqs = defaultdict(int)
    ipas = [_.strip("/") for _ in dataset.values()]
    unique_ipas = set(ipas)
    for ipa in unique_ipas:
        for char in ipa:
            freqs[char] += 1
    return freqs

In [40]:
freqs_es = get_phone_symbols_freq(dataset_ja)
# Sorted by freq number (d[1]) descendent (reverse=True)
distribution_es = dict(sorted(freqs_es.items(), key=lambda d: d[1], reverse=True))
df_es = pd.DataFrame.from_dict(distribution_es, orient='index')

In [41]:
df_es.head()

,0
ɯ,241499
i,198910
a,195490
o,171114
k,133252


#### 🧙🏼‍♂️ Ejercicio: Encontrar homófonos (palabras con el mismo sonido pero distinta ortografía) para el español

- Ejemplos: Casa-Caza, Vaya-Valla

In [44]:
from collections import Counter

transcription_counts = Counter(dataset_es_mx.values())
duplicated_transcriptions = [
    transcription for transcription, freq in transcription_counts.items() if freq > 1
]

for ipa in duplicated_transcriptions[-10:]:
    words = [
        word for word, transcription in dataset_es_mx.items() if transcription == ipa
    ]
    rprint(f"{ipa} => {words}")

/bexetas/ => ['vegetas', 'vejetas']

/bexete/ => ['vegete', 'vejete']

/bexetes/ => ['vegetes', 'vejetes']

/bexes/ => ['vejes', 'vejez']

/beɾas/ => ['veras', 'veraz']

/bise/ => ['vice', 'vise']

/bises/ => ['vices', 'vises']

/biβas/ => ['vivas', 'vivaz']

/bos/ => ['vos', 'voz']

/ʝak/ => ['yac', 'yak']

In [ ]:
Counter?

#### Obteniendo todos los datos

In [45]:
lang_codes = {
    "ar": "Arabic (Modern Standard)",
    "de": "German",
    "en_UK": "English (Received Pronunciation)",
    "en_US": "English (General American)",
    "eo": "Esperanto",
    "es_ES": "Spanish (Spain)",
    "es_MX": "Spanish (Mexico)",
    "fa": "Persian",
    "fi": "Finnish",
    "fr_FR": "French (France)",
    "fr_QC": "French (Québec)",
    "is": "Icelandic",
    "ja": "Japanese",
    "jam": "Jamaican Creole",
    "km": "Khmer",
    "ko": "Korean",
    "ma": "Malay (Malaysian and Indonesian)",
    "nb": "Norwegian Bokmål",
    "nl": "Dutch",
    "or": "Odia",
    "ro": "Romanian",
    "sv": "Swedish",
    "sw": "Swahili",
    "tts": "Isan",
    "vi_C": "Vietnamese (Central)",
    "vi_N": "Vietnamese (Northern)",
    "vi_S": "Vietnamese (Southern)",
    "yue": "Cantonese",
    "zh_hans": "Mandarin (Simplified)",
    "zh_hant": "Mandarin (Traditional)"
}
iso_lang_codes = list(lang_codes.keys())

In [48]:
def get_corpora() -> dict:
    """Download corpora from ipa-dict github

    Given a list of iso lang codes download available datasets.

    Returns
    -------
    dict
        Lang codes as keys and dictionary with words-transcriptions
        as values
    """
    return {
        code: parse_response(download_ipa_corpus(code))
         for code in iso_lang_codes
        }

In [49]:
corpora = get_corpora()

#### Sistema de búsqueda (naïve)

In [50]:
def get_formated_string(code: str, name: str):
    return f"[b]{name}[/b]\n[yellow]{code}"

In [53]:
rprint(
    Panel(Text("Representación fonética de palabras", style="bold", justify="center"))
)
rendable_langs = [
    Panel(get_formated_string(code, lang), expand=True)
    for code, lang in lang_codes.items()
]
rprint(Columns(rendable_langs))

lang = input("lang>> ")
rprint(f"Selected language: {lang_codes[lang]}") if lang else rprint("Adios 👋🏼")
while lang:
    dataset = corpora[lang]
    query = input(f"  [{lang}]word>> ")
    results = get_ipa_transcriptions(query, dataset)
    print(query, " | ", ", ".join(results))
    while query:
        query = input(f"  [{lang}]word>> ")
        if query:
            results = get_ipa_transcriptions(query, dataset)
            rprint(query, " | ", ", ".join(results))
    lang = input("lang>> ")
    rprint(f"Selected language: [yellow]{lang_codes[lang]}[/]") if lang else rprint(
        "Adios 👋🏼"
    )

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                       Representación fonética de palabras                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Arabic (Modern Standard)   │ │ German                           │ │ English (Received Pronunciation) │
│ ar                         │ │ de                               │ │ en_UK                            │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ English (General American) │ │ Esperanto                        │ │ Spanish (Spain)                  │
│ en_US                      │ │ eo                               │ │ es_ES                            │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Spanish (Mexico)           │ │ Persian                          │ │ Finnish                          │
│ es_MX                      │ │ fa                               │ │ fi                               │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ French (France)            │ │ French (Québec)                  │ │ Icelandic                        │
│ fr_FR                      │ │ fr_QC                            │ │ is                               │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Japanese                   │ │ Jamaican Creole                  │ │ Khmer                            │
│ ja                         │ │ jam                              │ │ km                               │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Korean                     │ │ Malay (Malaysian and Indonesian) │ │ Norwegian Bokmål                 │
│ ko                         │ │ ma                               │ │ nb                               │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Dutch                      │ │ Odia                             │ │ Romanian                         │
│ nl                         │ │ or                               │ │ ro                               │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Swedish                    │ │ Swahili                          │ │ Isan                             │
│ sv                         │ │ sw                               │ │ tts                              │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Vietnamese (Central)       │ │ Vietnamese (Northern)            │ │ Vietnamese (Southern)            │
│ vi_C                       │ │ vi_N                             │ │ vi_S                             │
╰────────────────────────────╯ ╰──────────────────────────────────╯ ╰──────────────────────────────────╯
╭────────────────────────────╮ ╭──────────────────────────────────╮ ╭──────────────────────────────────╮
│ Cantonese                  │ │ Mandarin (Simplified)            │ │ Mandarin (Traditional)           │
│ yue     

lang>>  en_US


Selected language: English (General American)

  [en_US]word>>  hello


...

hotel  |  /hoʊˈtɛɫ/

  [en_US]word>>  merequetengue


merequetengue  |

  [en_US]word>>  
lang>>  


Adios 👋🏼

#### 👩‍🔬 Ejercicio: *[Ortographic Depth](https://en.wikipedia.org/wiki/Orthographic_depth)*

Algunas lenguas se escriben como suenan y otras no. Calcula el promedio de los *ratios* (`len(chars) / len(phones)`) para todas las lenguas en el corpus. La salida debe verse más o menos así:

```
Φ Spanish (Spain) = ??
Φ Spanish (Mexico) = ??
Φ English (General American) = ??
```

Dónde `??` deberá mostrar el ratio (`Φ`) calculado

In [54]:
import numpy as np


def calculate_orthographic_depth(dataset):
    ratios = []
    for word, ipa in dataset.items():
        clean_ipa = ipa.strip("/").replace("ˈ", "")
        # Ratio: Letras por Sonido
        if len(clean_ipa) > 0:
            ratios.append(len(word) / len(clean_ipa))
    return np.mean(ratios)


for iso, dataset in corpora.items():
    rprint(f"Φ {lang_codes[iso]} = {calculate_orthographic_depth(dataset):.3f}")



Φ Arabic (Modern Standard) = 0.488

Φ German = 0.908

Φ English (Received Pronunciation) = 1.054

Φ English (General American) = 1.056

Φ Esperanto = 0.965

...

Φ Vietnamese (Northern) = 0.636

Φ Vietnamese (Southern) = 0.643

Φ Cantonese = 0.170

Φ Mandarin (Simplified) = 0.164

Φ Mandarin (Traditional) = 0.164

##### ¿Pregunta?: ¿Qué lengua usarías si quieres reducir costos en un LLM?

#### Ejemplo: Fabricando rimas

In [55]:
def get_rhyming_patterns(sentence: str, dataset: dict) -> dict[str, list]:
    words = sentence.split()
    word_ipa_map = {}
    for word in words:
        ipa_transcriptions = get_ipa_transcriptions(word, dataset)
        # Remove "/" char from transcriptions
        word_ipa_map.update({word: [_.strip("/") for _ in ipa_transcriptions]})

    rhyming_patterns = defaultdict(list)
    for word, ipas in word_ipa_map.items():
        for ipa in ipas:
            # Getting last 2 elements of the ipa representation
            pattern = ipa[-2:]
            rhyming_patterns[pattern].append(word)
    return rhyming_patterns

In [56]:
def display_rhyming_patterns(patterns: dict[str, list]) -> None:
    for pattern, words in patterns.items():
        if len(set(words)) > 1:
            print(f"{pattern}:: {', '.join(words)}")

#### Testing

```
ɣo:: juego, fuego
on:: con, corazón
ʎa:: brilla, orilla
```

In [59]:
sentence = "cuando juego con fuego siento como brilla la orilla de mi corazón"

dataset = corpora.get("es_MX")
rhyming_words = get_rhyming_patterns(sentence, dataset)
display_rhyming_patterns(rhyming_words)

ɣo:: juego, fuego
on:: con, corazón
ʎa:: brilla, orilla


#### Material extra (fonética)

In [ ]:
# apt-get install -y espeak
!sudo pacman -S espeak-ng

In [66]:
!espeak  --voices

Pty Language       Age/Gender VoiceName          File                 Other Languages
 5  af              --/M      Afrikaans          gmw/af               
 5  am              --/M      Amharic            sem/am               
 5  an              --/M      Aragonese          roa/an               
 5  ar              --/M      Arabic             sem/ar               
 5  as              --/M      Assamese           inc/as               
 5  az              --/M      Azerbaijani        trk/az               
 5  ba              --/M      Bashkir            trk/ba               
 5  be              --/M      Belarusian         zle/be               
 5  bg              --/M      Bulgarian          zls/bg               
 5  bn              --/M      Bengali            inc/bn               
 5  bpy             --/M      Bishnupriya_Manipuri inc/bpy              
 5  bs              --/M      Bosnian            zls/bs               
 5  ca              --/M      Catalan            roa/ca     

In [70]:
!espeak  -v roa/es-419 "Camara banda ya se la saben celulares y carteras" --ipa

kamˈaɾa βˈanda ʝˈa se la sˈaβen sˌelulˈaɾes i kaɾtˈeɾas


## Morfología

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Flexi%C3%B3nGato-svg.svg/800px-Flexi%C3%B3nGato-svg.svg.png" width=200></center>

> De <a href="//commons.wikimedia.org/wiki/User:KES47" class="mw-redirect" title="User:KES47">KES47</a> - <a href="//commons.wikimedia.org/wiki/File:Flexi%C3%B3nGato.png" title="File:FlexiónGato.png">File:FlexiónGato.png</a> y <a href="//commons.wikimedia.org/wiki/File:Nuvola_apps_package_toys_svg.svg" title="File:Nuvola apps package toys svg.svg">File:Nuvola apps package toys svg.svg</a>, <a href="http://www.gnu.org/licenses/lgpl.html" title="GNU Lesser General Public License">LGPL</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=27305101">Enlace</a>

El análisis morfológico es la determinación de las partes que componen la palabra y su representación lingüística, es una especie de etiquetado

Los elementos morfológicos son analizados para:

- Determinar la función morfológica de las palabras
- Hacer filtrado y pre-procesamiento de text

### Análisis morfológico basado en reglas

Recordemos que podemos hacer un analizador morfológico haciendo uso de un transductor que vaya leyendo y haciendo transformaciones en una cadena. Formalmente:

* $Q = \{q_0, \ldots, q_T\}$ conjunto finito de estados.
* $\Sigma$ es un alfabeto de entrada.
* $q_0 \in Q$ es el estado inicial.
* $F \subseteq Q$ es el conjunto de estados finales.

Un transductor es una 6-tupla $T = (Q, \Sigma, \Delta, q_0, F, \sigma)$ tal que

* $\Delta$ es un alfabeto de salida teminal
* $\Sigma$ es un alfabeto de entrada no terminal
* $\sigma: Q \times \Sigma \times \Delta \longrightarrow Q$ función de transducción

#### Ejemplo: Parsing con expresiones regulares

Con fines de prácticidad vamos a _imitar_ el comportamiento de un transductor utilizando el modulo de python `re`

La estructura del sustantivo en español es:

` BASE+AFIJOS (marcas flexivas)   --> Base+DIM+GEN+NUM`

In [ ]:
palabras = [
    'niño',
    'niños',
    'niñas',
    'niñitos',
    'gato',
    'gatos',
    'gatitos',
    'perritos',
    'paloma',
    'palomita',
    'palomas',
    'flores',
    'flor',
    'florecita',
    'lápiz',
    'lápices',
    # 'chiquitititititos',
    #'curriculum', # curricula
    #'campus', # campi
]

In [ ]:
def morph_parser_rules(words: list[str]) -> list[str]:
    """Aplica reglas morfológicas a una lista de palabras para realizar
    un análisis morfológico.

    Parameters:
    ----------
    words : list of str
        Lista de palabras a las que se les aplicarán las reglas morfológicas.

    Returns:
    -------
    list of str
        Una lista de palabras después de aplicar las reglas morfológicas.
    """

    # Lista para guardar las palabras parseadas
    morph_parsing = []

    # Reglas que capturan ciertos morfemas
    # {ecita, itos, as, os}
    for w in words:
        # ecit -> DIM
        R0 = re.sub(r"([^ ]+)ecit([a|o|as|os])", r"\1-DIM\2", w)
        # it -> DIM
        R1 = re.sub(r"([^ ]+)it([a|o|as|os])", r"\1-DIM\2", R0)
        # a(s) -> FEM
        R2 = re.sub(r"([^ ]+)a(s)", r"\1-FEM\2", R1)
        # a -> FEM
        R3 = re.sub(r"([^ ]+)a\b", r"\1-FEM", R2)
        # o(s) -> MSC
        R4 = re.sub(r"([^ ]+)o(s)", r"\1-MSC\2", R3)
        # o .> MSC
        R5 = re.sub(r"([^ ]+)o\b", r"\1-MSC", R4)
        # es -> PL
        R6 = re.sub(r"([^ ]+)es\b", r"\1-PL", R5)
        # s -> PL
        R7 = re.sub(r"([^ ]+)s\b", r"\1-PL", R6)
        # Sustituye la c por z cuando es necesario
        parse = re.sub(r"c-", r"z-", R7)

        # Guarda los parseos
        morph_parsing.append(parse)
    return morph_parsing

In [ ]:
def prettify_tags(word: str) -> str:
    tags = {
        "DIM": "[b yellow]DIM[/]",
        "FEM": "[b green]FEM[/]",
        "MSC": "[b magenta]MSC[/]",
        "PL": "[b blue]PL[/]",
    }
    for tag, pretty_tag in tags.items():
        word = word.replace(tag, pretty_tag)
    return word

In [ ]:
morph_parsing = morph_parser_rules(palabras)
for palabra, parseo in zip(palabras, morph_parsing):
    rprint(palabra, "-->", prettify_tags(parseo))

#### Preguntas 🤔
- ¿Qué pasa con las reglas en lenguas donde son más comunes los prefijos y no los sufijos?
- ¿Cómo podríamos identificar características de las lenguas?

#### Herramientas para hacer sistemas de análisis morfológico basados en reglas

- [Apertium](https://en.wikipedia.org/wiki/Apertium)
- [Foma](https://github.com/mhulden/foma/tree/master)
- [Helsinki Finite-State Technology](https://hfst.github.io/)
- Ejemplo [proyecto](https://github.com/apertium/apertium-yua) de analizador morfológico de Maya Yucateco
- Ejemplo normalizador ortográfico del [Náhuatl](https://github.com/ElotlMX/py-elotl/tree/master)


También se pueden utilizar diferentes métodos de aprendizaje de máquina para realizar análisis/generación morfológica. En los últimos años ha habido un shared task de [morphological reinflection](https://github.com/sigmorphon/2023InflectionST) para poner a competir diferentes métodos

### Segmentación morfológica

#### Corpus: [SIGMORPHON 2022 Shared Task on Morpheme Segmentation](https://github.com/sigmorphon/2022SegmentationST/tree/main)

- Shared task donde se buscaba convertir las palabras en una secuencia de morfemas
- Dividido en dos partes:
    - Segmentación a nivel de palabras (nos enfocaremos en esta)
    - Segmentación a nivel oraciones

#### Track: words

| word class | Description                      | English example (input ==> output)     |
|------------|----------------------------------|----------------------------------------|
| 100        | Inflection only                  | played ==> play @@ed                   |
| 010        | Derivation only                  | player ==> play @@er                   |
| 101        | Inflection and Compound          | wheelbands ==> wheel @@band @@s        |
| 000        | Root words                       | progress ==> progress                  |
| 011        | Derivation and Compound          | tankbuster ==> tank @@bust @@er        |
| 110        | Inflection and Derivation        | urbanizes ==> urban @@ize @@s          |
| 001        | Compound only                    | hotpot ==> hot @@pot                   |
| 111        | Inflection, Derivation, Compound | trackworkers ==> track @@work @@er @@s

#### Explorando el corpus

In [ ]:
response = r.get("https://raw.githubusercontent.com/sigmorphon/2022SegmentationST/main/data/spa.word.test.gold.tsv")
response.text[:100]

In [ ]:
raw_data = response.text.split("\n")
raw_data[-2]

In [ ]:
element = raw_data[2].split("\t")
element

In [ ]:
element[1].split()

In [ ]:
LANGS = {
    "ces": "Czech",
    "eng": "English",
    "fra": "French",
    "hun": "Hungarian",
    "spa": "Spanish",
    "ita": "Italian",
    "lat": "Latin",
    "rus": "Russian",
}
CATEGORIES = {
    "100": "Inflection",
    "010": "Derivation",
    "101": "Inflection, Compound",
    "000": "Root",
    "011": "Derivation, Compound",
    "110": "Inflection, Derivation",
    "001": "Compound",
    "111": "Inflection, Derivation, Compound"
}

In [ ]:
def get_track_files(lang: str, track: str = "word") -> list[str]:
    """Genera una lista de nombres de archivo del shared task

    Con base en el idioma y el track obtiene el nombre de los archivos
    para con información reelevante para hacer análisis estadístico.
    Esto es archivos .test y .dev

    Parameters:
    ----------
    lang : str
        Idioma para el cual se generarán los nombres de archivo.
    track : str, optional
        Track del shared task de donde vienen los datos (por defecto es "word").

    Returns:
    -------
    list[str]
        Una lista de nombres de archivo generados para el idioma y la pista especificados.
    """
    return [
        f"{lang}.{track}.test.gold",
        f"{lang}.{track}.dev",
    ]

In [ ]:
def get_raw_corpus(files: list) -> list:
    """Descarga y concatena los datos de los archivos tsv desde una URL base.

    Parameters:
    ----------
    files : list
        Lista de nombres de archivos (sin extensión) que se descargarán
        y concatenarán.

    Returns:
    -------
    list
        Una lista que contiene los contenidos descargados y concatenados
        de los archivos tsv.
    """
    result = []
    for file in files:
        print(f"Downloading {file}.tsv", end=" ")
        response = r.get(f"https://raw.githubusercontent.com/sigmorphon/2022SegmentationST/main/data/{file}.tsv")
        print(f"status={response.status_code}")
        lines = response.text.split("\n")
        result.extend(lines[:-1])
    return result

In [ ]:
def raw_corpus_to_dataframe(corpus_list: list, lang: str) -> pd.DataFrame:
    """Convierte una lista de datos de corpus en un DataFrame

    Parameters:
    ----------
    corpus_list : list
        Lista de líneas del corpus a convertir en DataFrame.
    lang : str
        Idioma al que pertenecen los datos del corpus.

    Returns:
    -------
    pd.DataFrame
        Un DataFrame de pandas que contiene los datos del corpus procesados.
    """
    data_list = []
    for line in corpus_list:
        try:
            word, tagged_data, category = line.split("\t")
        except ValueError:
            # Caso donde no existe la categoria
            word, tagged_data = line.split("\t")
            category = "NOT_FOUND"
        morphemes = tagged_data.split()
        data_list.append(
            {"words": word, "morph": morphemes, "category": category, "lang": lang}
        )
    df = pd.DataFrame(data_list)
    df["word_len"] = df["words"].apply(lambda x: len(x))
    df["morph_count"] = df["morph"].apply(lambda x: len(x))
    return df

In [ ]:
files = get_track_files("spa")
raw_spa = get_raw_corpus(files)
df = raw_corpus_to_dataframe(raw_spa, lang="spa")

In [ ]:
df.head(20)

#### Análisis cuantitativo para el Español

In [ ]:
print("Total unique words:", len(df["words"].unique()))
df["category"].value_counts().head(30)

In [ ]:
df["word_len"].mean()

In [ ]:
plt.hist(df['word_len'], bins=10, edgecolor='black')
plt.xlabel('Word Length')
plt.ylabel('Frequency')
plt.title('Word Length Distribution')
plt.show()

In [ ]:
def plot_histogram(df, kind, lang):
    """Genera un histograma de frecuencia para una columna específica
    en un DataFrame.

    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame que contiene los datos para generar el histograma.
    kind : str
        Nombre de la columna para la cual se generará el histograma.
    lang : str
        Idioma asociado a los datos.

    Returns:
    -------
    None
        Esta función muestra el histograma usando matplotlib.
    """
    counts = df[kind].value_counts().head(30)
    plt.bar(counts.index, counts.values)
    plt.xlabel(kind)
    plt.ylabel('Frequency')
    plt.title(f'{kind} Frequency Graph for {lang}')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_histogram(df, "category", "spa")

#### Ejemplo 🥑: Adivina, adivinador, probabilidad del tipo de morfemas

- No todos los afijos son iguales. Algunos sirven para conjugar (Flexivos) y otros para crear conceptos nuevos (Derivativos).
- Aprovechando que el dataset de SIGMORPHON nos dice si una palabra es de tipo `Inflection` (100) o `Derivation` (010) calculemos qué tan probable es que un morfema sea flexivo o derivativo.


In [ ]:
def analyze_morpheme_types(df: pd.DataFrame) -> pd.DataFrame:
    inflection_counts = defaultdict(int)
    derivation_counts = defaultdict(int)
    total_counts = defaultdict(int)

    for _, row in df.iterrows():
        morphemes = row['morph'] 
        category = row['category']

        # Con base en SIGMORPHON:
        # Pos 0: Inflection, Pos 1: Derivation
        is_inflection = category[0] == '1'
        is_derivation = category[1] == '1'

        for morph in morphemes:
            clean_morph = morph.replace("@@", "")

            # Ignoramos morfemas muy cortos o vacíos
            if len(clean_morph) < 2:
                continue

            total_counts[clean_morph] += 1
            if is_inflection:
                inflection_counts[clean_morph] += 1
            if is_derivation:
                derivation_counts[clean_morph] += 1

    results = []
    for morph, total in total_counts.items():
        if total > 5:
            p_inflection = inflection_counts[morph] / total
            p_derivation = derivation_counts[morph] / total
            results.append({
                'morfema': morph,
                'total_freq': total,
                'prob_flexion': round(p_inflection, 3),
                'prob_derivacion': round(p_derivation, 3)
            })

    return pd.DataFrame(results).sort_values(by='total_freq', ascending=False)

In [ ]:
morph_stats = analyze_morpheme_types(df)

#### Los morfemas más "Gramaticales" (Flexivos)

Deberíamos ver terminaciones verbales y plurales.

In [ ]:
rprint("Top Morfemas Flexivos:")
rprint(morph_stats.sort_values(by=['prob_flexion', "total_freq"], ascending=False).head(10))

#### Los morfemas más "Léxicos" (Derivativos)

Deberíamos ver sufijos que cambian sustantivos a adjetivos, etc.

In [ ]:
rprint("Top Morfemas Derivativos:")
rprint(morph_stats.sort_values(by=['prob_derivacion', "total_freq"], ascending=False).head(10))

#### Morfosintaxis

- Etiquetas que hacen explícita la funcion gramatical de las palabras en una oración
- Determina la función de la palabra dentro la oración (por ello se le llama Partes del Discurso)
- Se le conoce tambien como **Análisis morfosintáctico**: es el puente entre la estructura de las palabras y la sintaxis
- Permiten el desarrollo de herramientas de NLP más avanzadas
- El etiquetado es una tarea que se puede abordar con técnicas secuenciales, por ejemplo, HMMs, CRFs, Redes neuronales

<center><img src="https://byteiota.com/wp-content/uploads/2021/01/POS-Tagging.jpg" height=500 width=500></center

#### Ejemplo

> El gato negro rie malvadamente

- El - DET
- gato - NOUN
- negro - ADJ
- ríe - VER

<center><img src="https://i.pinimg.com/originals/0e/f1/30/0ef130b255ea704625b2ad473701dee5.gif"></center

### Etiquetado POS usando Hidden Markov Models

In [1]:
# TODO

### Etiquetado POS usando Conditional Random Fields (CRFs)

- Modelo de gráficas **no dirigido**. Generaliza los *HMM*
    - Adiós a la *Markov assuption*
    - Podemos tener cualquier dependencia que queramos entre nodos
    - Nos enfocaremos en un tipo en concreto: *LinearChain-CRFs* ¡¿Por?!

<center><img width=300 src="https://i.kym-cdn.com/entries/icons/original/000/032/676/Unlimited_Power_Banner.jpg"></center>


- Modela la probabilidad **condicional** $P(Y|X)$
    - Modelo discriminativo
    - Probabilidad de un estado oculto dada **toda** la secuecia de entrada
![homer](https://media.tenor.com/ul0qAKNUm2kAAAAd/hiding-meme.gif)

- Captura mayor **número de dependencias** entre las palabras y captura más características
    - Estas se definen en las *feature functions* 🙀
- El entrenamiento se realiza aplicando gradiente decendente y optimización con algoritmos como [L-BFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS)


<center><img src="https://iameo.github.io/images/gradient-descent-400.gif"></center>


$P(\overrightarrow{y}|\overrightarrow{x}) = \frac{1}{Z} \displaystyle\prod_{i=1}^N exp\{w^T ⋅ \phi(y_{i-1}, y_i, \overrightarrow{x}, i)\}$

Donde:
- $\overrightarrow{y}$ = Etiquetas POS
- $\overrightarrow{x}$ = Palabras en una oración
- $w^T$ = Vector de pesos a aprender
- $\phi$ = Vector de *Features*
    - Calculado con base en un conjunto de *feature functions*
- $i$ = la posición actual en la oración
- $Z$ = factor de normalización

![](https://aman.ai/primers/ai/assets/conditional-random-fields/Conditional_Random_Fields.png)

Tomado de http://www.davidsbatista.net/blog/2017/11/13/Conditional_Random_Fields/

#### Feature functions

$\phi(y_{i-1}, y_i, \overrightarrow{x}, i)$

- Parte fundamental de los CRFs
- Cuatro argumentos:
    - Todos los datos observables $\overrightarrow{x}$ (conectar $x$ con cualquier $y$)
    - El estado oculto anterior $y_{i-1}$
    - El estado oculto actual $y_i$
    - El index del timestamp $i$
        - Cada feature list puede tener diferentes formas

- Aquí es donde está la flexibilidad del modelo
- Tantas *features* como queramos, las que consideremos que pueden ayudar a que el modelo tenga un mejor desempeño
    - Íntimamente ligadas a la lengua. Para mejor desempeño se debe hacer un análisis de sus características.
- Ejemplo:

```python
[
    "word.lower()",
    "EOS",
    "BOS",
    "pre-word",
    "nxt-word",
    "word-position",
    ...
]
```

### Implementación de CRFs

In [ ]:
!uv add nltk scikit-learn sklearn-crfsuite

#### Obteniendo otro corpus más

In [ ]:
# Descargando el corpus cess_esp: https://www.nltk.org/book/ch02.html#tab-corpora
nltk.download('cess_esp')

In [ ]:
# Cargando oraciones
corpora = cess_esp.tagged_sents()

In [ ]:
rprint(corpora[1])

In [ ]:
def get_tags_map() -> dict:
    tags_raw = r.get(
        "https://gist.githubusercontent.com/vitojph/39c52c709a9aff2d1d24588aba7f8155/raw/af2d83bc4c2a7e2e6dbb01bd0a10a23a3a21a551/universal_tagset-ES.map"
    ).text.split("\n")
    tags_map = {line.split("\t")[0].lower(): line.split("\t")[1] for line in tags_raw}
    return tags_map


def map_tag(tag: str, tags_map=get_tags_map()) -> str:
    return tags_map.get(tag.lower(), "N/F")


def parse_tags(corpora: list[list[tuple]]) -> list[list[tuple]]:
    result = []
    for sentence in corpora:
        result.append([(word, map_tag(tag)) for word, tag in sentence])
    return result

In [ ]:
corpora = parse_tags(corpora)

In [ ]:
rprint(corpora[0])

#### Feature lists

In [ ]:
def word_to_features(sent, i):
    word = sent[i][0]
    features = {
        "word.lower()": word.lower(),
        "word[-3:]": word[-3:],
        "word[-2:]": word[-2:],
        "prefix_1": word[:1],
        "prefix_2": word[:2],
        "word.isupper()": word.isupper(),
        "word.istitle()": word.istitle(),
        "word.isdigit()": word.isdigit(),
        "word_len": len(word),
    }
    if i > 0:
        prev_word = sent[i - 1][0]
        features.update(
            {
                "prev_word.lower()": prev_word.lower(),
                "prev_word.istitle()": prev_word.istitle(),
            }
        )
    else:
        # Beginning of sentence
        features["BOS"] = True
    return features


# Extract features and labels
def sent_to_features(sent) -> list:
    return [word_to_features(sent, i) for i in range(len(sent))]


def sent_to_labels(sent) -> list:
    return [label for token, label in sent]

In [ ]:
# ¿Cuantas oraciones tenemos disponibles?
len(corpora)

In [ ]:
# Preparando datos para el CRF
X = [[word_to_features(sent, i) for i in range(len(sent))] for sent in corpora]
y = [[pos for _, pos in sent] for sent in corpora]

In [ ]:
# Exploración de data estructurada
rprint(X[0])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
assert len(X_train) + len(X_test) == len(corpora), "Something wrong with my split :("
assert len(y_train) + len(y_test) == len(corpora), "Something wrong with my split :("

In [ ]:
# Initialize and train the CRF tagger: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html
crf = CRF(
    algorithm="lbfgs",
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True,
)
try:
    crf.fit(X_train, y_train)
except AttributeError as e:
    print(e)

In [ ]:
y_pred = crf.predict(X_test)

# Flatten the true and predicted labels
y_test_flat = [label for sent_labels in y_test for label in sent_labels]
y_pred_flat = [label for sent_labels in y_pred for label in sent_labels]

# Evaluate the model
report = classification_report(y_true=y_test_flat, y_pred=y_pred_flat)
rprint(report)

# Tarea 1: Exploración de Niveles del lenguaje 🔭

### FECHA DE ENTREGA: 2 de Marzo 2026 at 11:59pm

### Fonética

1. Con base en el sistema de búsqueda visto en clase que recibe una palabra ortográfica y devuelve sus transcripciones fonológicas, proponga una solución para los casos en que la palabra buscada no se encuentra en el lexicón/diccionario.
    - ¿Cómo devolver o **aproximar** su transcripción fonológica?
    - Reutiliza el sistema de búsqueda visto en clase y mejóralo con esta funcionalidad.
    - Muestra al menos tres ejemplos

### Morfología

2. Elige tres lenguas del corpus que pertenezcan a familias lingüísticas distintas
   - Ejemplo: `spa` (Romance), `eng` (Germánica), `hun` (Urálica)
   - Para cada una de las tres lenguas calcula y compara:
       -  **Ratio morfemas / palabra**: El promedio de morfemas que componen las palabras
        -  **Indicé de Flexión / Derivación**: Del total de morfemas, ¿Qué porcentaje son etiquetas de flexión (`100`) y cuáles de derivación (`010`)?
3. Visualización
    - Genera una figura con **subplots** para comparar las lenguas lado a lado.
    - *Plot 1*: Distribución de la longitud de los morfemas
    - *Plot 2*: Distribución de las categorías (flexión, derivación, raíz, etc.)
4. Con base en esta información, responde la pregunta: *¿Cuál de las tres lenguas se comporta más como una lengua aglutinante y cuál como una lengua aislante?*
    - Justifica tu respuesta usando tus métricas y figuras

### EXTRA:

- Genera la [matriz de confusión](https://en.wikipedia.org/wiki/Confusion_matrix) para el etiquetador CRFs visto en clase
- Observando las etiquetas donde el modelo falló responde las preguntas:
    - ¿Por qué crees que se confundió?
    - ¿Es un problema de ambigüedad léxica (la palabra tiene múltiples etiquetas)?
    - ¿Qué *features* añadirías para solucionarlo?